In [2]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [4]:
import os
import zipfile

local_zip = '/gdrive/My Drive/NEU-DET_new.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./')
zip_ref.close()

In [5]:
train_crazing_dir = os.path.join('/content/NEU-DET/train/images/crazing')
train_inclusion_dir = os.path.join('/content/NEU-DET/train/images/inclusion/')
train_patches_dir = os.path.join('/content/NEU-DET/train/images/patches/')
train_pitted_surface_dir = os.path.join('/content/NEU-DET/train/images/pitted_surface/')
train_rolledin_scale_dir = os.path.join('/content/NEU-DET/train/images/rolled-in_scale/')
train_scratches_dir = os.path.join('/content/NEU-DET/train/images/scratches/')

In [7]:
train_crazing_names = os.listdir(train_crazing_dir)
train_inclusion_names = os.listdir(train_inclusion_dir)
train_patches_names = os.listdir(train_patches_dir)
train_pitted_surface_names = os.listdir(train_pitted_surface_dir)
train_rolledin_scale_names = os.listdir(train_rolledin_scale_dir)
train_scratches_names = os.listdir(train_scratches_dir)

In [8]:
print('total training crazing images:', len(train_crazing_names))
print('total training crazing images:', len(train_inclusion_names))
print('total training crazing images:', len(train_patches_names))
print('total training crazing images:', len(train_pitted_surface_names))
print('total training crazing images:', len(train_rolledin_scale_names))
print('total training crazing images:', len(train_scratches_names))

total training crazing images: 240
total training crazing images: 240
total training crazing images: 240
total training crazing images: 240
total training crazing images: 240
total training crazing images: 240


In [9]:
from keras.applications.mobilenet import MobileNet
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

Using TensorFlow backend.


In [10]:
input_shape = (224, 224, 3)
num_classes = 6
conv_size = 32

In [11]:
MobileNet_model = MobileNet(include_top=False, input_shape=input_shape)

17227776/17225924 [==============================] - 2s 0us/step


In [12]:
# add custom Layers
x = MobileNet_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
Custom_Output = Dense(num_classes, activation='softmax')(x)


In [13]:
model = Model(inputs = MobileNet_model.input, outputs = Custom_Output)
        
# compile the model
model.compile(loss='categorical_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Rescaling images
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    directory = '/content/NEU-DET/train/images/',
    target_size = (224, 224),
    batch_size = 60,
    class_mode = 'categorical',
    subset='training'
)

Found 1152 images belonging to 6 classes.


In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Rescaling images
train_datagen_y = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

train_generator_y = train_datagen_y.flow_from_directory(
    directory = '/content/NEU-DET/train/images/',
    target_size = (224, 224),
    batch_size = 60,
    class_mode = 'categorical',
    subset='validation'
)

Found 288 images belonging to 6 classes.


In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=num_epochs,
                    verbose=1,
                    validation_split=0.1)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=8,
    epochs=15,
    verbose=1,
)

Epoch 1/15
8/8 [==============================] - 25s 3s/step - loss: 0.4256 - accuracy: 0.8396
Epoch 2/15
8/8 [==============================] - 8s 1s/step - loss: 0.1245 - accuracy: 0.9653
Epoch 3/15
8/8 [==============================] - 9s 1s/step - loss: 0.1292 - accuracy: 0.9688
Epoch 4/15
8/8 [==============================] - 8s 978ms/step - loss: 0.0969 - accuracy: 0.9676
Epoch 5/15
8/8 [==============================] - 9s 1s/step - loss: 0.1007 - accuracy: 0.9792
Epoch 6/15


In [ ]:
# Testing on images